In [55]:
import seaborn as sns
import basefunction.anass_load as at
import basefunction.vBaseFunctions3 as vbf
import bk.compute
import bk.signal
import bk.plot
import bk.load
import basefunction.assembly as assembly
import numpy as np
import pandas as pd
import neuroseries as nts
import scipy.io

import pickle

import time
import matplotlib.pyplot as plt

plt.rcParams['svg.fonttype'] = 'none'
plt.style.use('ggplot')

%matplotlib qt

In [17]:
def main(base_folder, local_path, *args, **kwargs):
    bk.load.current_session_linux(base_folder, local_path)
    states = bk.load.states()
    chan = {'left': bk.load.bla_channels()['left'],
            'right': bk.load.bla_channels()['right']}
    if np.isnan(chan['left']) or np.isnan(chan['right']):
        return False
    
    Cxy = {}
    for state in ['sws','Rem']:
        lfp = {}
        for side in ['left','right']: lfp[side] = bk.load.lfp_in_intervals(bk.load.bla_channels()[side],states[state])
        f,Cxy[state] = scipy.signal.coherence(lfp['left'],lfp['right'],nperseg=10_000,noverlap=5_000,fs = 1250)

    return f,Cxy

In [19]:
meta, batch = bk.load.batch(main)

100%|██████████| 61/61 [1:10:34<00:00, 69.41s/it]

Batch finished in 4234.295715808868
Some session were not processed correctly
['Rat08-20130720', 'Rat08-20130722', 'Rat09-20140407', 'Rat09-20140408', 'Rat09-20140409', 'Rat11-20150402', 'Rat11-20150403']
11.475409836065573 %


In [28]:
def merge(batch):
    Cxy = {'sws':[],
            'Rem':[]}
    for sess,b in batch.items():
        if not b:
            continue
        f = b[0]
        Cxy['sws'].append(b[1]['sws'])
        Cxy['Rem'].append(b[1]['Rem'])
    return f,Cxy

In [30]:
f,Cxy = merge(meta)

In [34]:
y = np.array(Cxy['sws'])

In [45]:
def plot(f,Cxy,st,ax = None):
    bk.plot.confidence_intervals(f,Cxy,st,ax)

In [57]:
fig,ax = plt.subplots(1,1)
plot(f,Cxy['sws'],'grey',ax)
plot(f,Cxy['Rem'],'orange',ax)
plt.xlim(0,120)
plt.ylim(0,1)

plt.ylabel('Coherence Cxy')
plt.xlabel('Frequency (Hz)')
plt.title('Coherence between Left and Right BLA')

Text(0.5, 1.0, 'Coherence between Left and Right BLA')